In [1]:
# Import the QICK drivers and auxiliary libraries
from qick import *
%pylab inline
import numpy as np

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
# Pyro to communicate with board
import Pyro4
from qick import QickConfig
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION=4

# Static IP proxy
ns_host = "172.24.106.21" # <-- SET IP ADDRESS
ns_port = 8888
proxy_name = "myqick"

# QICK object
ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)


QICK configuration:

	Board: RFSoC4x2

	Software version: 0.2.267
	Firmware timestamp: Wed Sep  6 18:49:29 2023

	Global clocks (MHz): tProcessor 409.600, RF reference 491.520

	2 signal generator channels:
	0:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 0, blk 0 is DAC_B
	1:	axis_signal_gen_v6 - envelope memory 65536 samples (6.667 us)
		fs=9830.400 MHz, fabric=614.400 MHz, 32-bit DDS, range=9830.400 MHz
		DAC tile 2, blk 0 is DAC_A

	2 readout channels:
	0:	axis_readout_v2 - configured by PYNQ
		fs=4423.680 MHz, decimated=552.960 MHz, 32-bit DDS, range=4423.680 MHz
		maxlen 16384 accumulated, 1024 decimated (1.852 us)
		triggered by output 7, pin 14, feedback to tProc input 0
		ADC tile 0, blk 0 is ADC_D
	1:	axis_readout_v2 - configured by PYNQ
		fs=4423.680 MHz, decimated=552.960 MHz, 32-bit DDS, range=4423.680 MHz
		maxlen 16384 accumulated, 1024 decimated (1.852 us)
		triggered by o

In [114]:
def gen_iq_data(sequence):
    fs_dac = 9830.4
    gain = 32766

    idata, qdata = np.array([]), np.array([])
    actual_times = []

    for params in sequence:
        # Extract parameters
        length = params[0]
        amp = params[1]

        # Calculate number of DAC sampling cycles and actual pulse time
        cycles = int(np.round((length/1e3) * fs_dac, 0))
        actual_times.append(cycles/fs_dac * 1e3)

        # Add pulses to idata and qdata
        if 0 < amp <= 1: # Producing output
            # Calculate amp_i and amp_q from phase and gain
            # phi = params[3]
            # dac_iq = np.round(np.e**(1j * phi * np.pi / 180), 10)
            # dac_i, dac_q = np.real(dac_iq), np.imag(dac_iq)
            dac_i, dac_q = 1, 1
            amp_i = int(gain * dac_i)
            amp_q = int(gain * dac_q)

            # Append appropriate number of amp_i and amp_q to idata and qdata
            idata = np.concatenate((idata, np.full(shape=cycles, fill_value=amp_i, dtype=int)))
            qdata = np.concatenate((qdata, np.full(shape=cycles, fill_value=amp_q, dtype=int)))
        elif amp == 0: # Not producing output
            # Append appropriate number of zeros to idata and qdata
            idata = np.concatenate((idata, np.zeros(cycles, dtype=int)))
            qdata = np.concatenate((qdata, np.zeros(cycles, dtype=int)))

    # Check lengths of idata and qdata match
    if len(idata) != len(qdata):
        raise Exception

    # Pad data so that array lengths are multiples of 16
    padding = 16 - (len(idata) % 16)
    idata = np.pad(idata,
                pad_width = (0, padding),
                mode = 'constant',
                constant_values = 0)
    qdata = np.pad(qdata,
                pad_width = (0, padding),
                mode = 'constant',
                constant_values = 0)

    # Calculate time that sequence occupies DAC
    t_busy = len(idata) / fs_dac # us

    # print(actual_times)
    # print(t_busy)

    return idata, qdata

In [115]:
sequence = [(50, 1.0, None, 45),
            # (20, 0),
            # (20, 1.0, None, 60),
            # (20, 0),
            # (50, 1.0, None, 0),
            ]

idata0, qdata0 = gen_iq_data(sequence)
print(idata0)

# idata1 = np.full(shape=9600, fill_value=24000, dtype=int)
# qdata1 = np.full(shape=9600, fill_value=24000, dtype=int)

[32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766.
 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766.
 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766.
 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766.
 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766.
 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766.
 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766.
 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766.
 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766.
 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766.
 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766.
 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766.
 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766.
 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766. 32766.
 32766

In [163]:
prog = QickProgram(soccfg)
prog.declare_gen(ch=1, nqz=1)
prog.add_envelope(ch=1, name="measure", idata=idata0, qdata=qdata0)
# self.add_envelope(ch=res_ch, name="test", idata=idata1, qdata=qdata1)

prog.set_pulse_registers(ch=1,
                         style="arb",
                         waveform="measure",
                         gain=32766,
                         freq=soccfg.freq2reg(200, gen_ch=0),
                         phase=soccfg.deg2reg(-45, gen_ch=0),
                         outsel="dds"
                         )
prog.pulse(ch=1)

prog.set_pulse_registers(ch=1,
                            style="const",
                            length=prog.us2cycles(0.1, gen_ch=0),
                            gain=23169, # 32766/sqrt(2)
                            freq=prog.freq2reg(200, gen_ch=0),
                            phase=prog.deg2reg(0, gen_ch=0),
                            # outsel='product'
                            )
prog.pulse(ch=1)

# self.set_pulse_registers(ch=self.cfg["res_ch"], style="arb", waveform="test", freq=freq)
# self.pulse(ch=self.cfg["res_ch"])

# prog.config_all(soc) # Configure for internal start
# soc.tproc.start() # Internal start
# print(prog)

In [164]:
print(prog)


// Program

  regwi 1, $22, 87381333;                       //freq = 87381333
  regwi 1, $23, 939524096;                      //phase = 3758096384
  bitwi 1, $23, $23 << 2;
  regwi 1, $25, 32766;                          //gain = 32766
  regwi 1, $24, 0;                              //addr = 0
  regwi 1, $26, 589855;                         //phrst| stdysel | mode | | outsel = 0b01001 | length = 31 
  regwi 1, $27, 0;                              //t = 0
  set 1, 1, $22, $23, $24, $25, $26, $27;       //ch = 1, pulse @t = $27
  regwi 1, $22, 87381333;                       //freq = 87381333
  regwi 1, $23, 0;                              //phase = 0
  regwi 1, $25, 23169;                          //gain = 23169
  regwi 1, $26, 589885;                         //phrst| stdysel | mode | | outsel = 0b01001 | length = 61 
  regwi 1, $27, 20;                             //t = 20
  set 1, 1, $22, $23, $0, $25, $26, $27;        //ch = 1, pulse @t = $27
